In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as lite

In [2]:
# Function to get table names from database
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables

In [3]:
# Get table names from database
conn = lite.connect('../unos.db')
tables = tables_in_sqlite_db(conn)
print(tables)

[]


In [5]:
# QUERY database to create dataframe
query = f"SELECT * FROM (SELECT *, strftime('%Y', {'TX_DATE'}) as Year FROM {'kidpan_data'}) data WHERE Year = '2010'"
df_10 = pd.read_sql_query(query,conn)

In [5]:
df_10.shape

(18077, 482)

In [6]:
# From data dictionary: 
#CONTIN_IV_DRUG_OLD_DON = DECEASED DONOR-HISTORY OF IV DRUG USE+RECENT 6MO USE VAR START DATE = 4/1/94
# HBA1C_PA_TCR = HBA1C AT LISTING (PA KP ONLY) -- VAR START DATE: 2/28/18
# FREE_DON = LIVING DONOR-FREE CARE AS SECONDARY PAYMENT TYPE -- VAR START DATE: 10/25/99
# DONATION_DON = LIVING DONOR-DONATION AS SECONDARY PAYMENT TYPE -- VAR START DATE: 10/25/99
# SELF_DON = LIVING DONOR-SELF AS SECONDARY PAYMENT TYPE -- VAR START DATE: 10/25/99 VAR END 1/1/07
df_10.isnull().sum().sort_values(ascending = False)

CONTIN_IV_DRUG_OLD_DON    18077
HBA1C_PA_TCR              18077
FREE_DON                  18077
DONATION_DON              18077
SELF_DON                  18077
                          ...  
DISTANCE                      0
DIAL_TRR                      0
ORGAN                         0
CMV_IGG                       0
index                         0
Length: 482, dtype: int64

In [7]:
# Drop columns in which all values are null (482 --> 431 Features)
df_10.dropna(axis = 1, how = 'all', inplace = True) 
df_10.shape

(18077, 431)

In [8]:
df_10.isnull().sum().sort_values(ascending=False)

TXLNG                      18076
C_PEPTIDE_PA_TCR           18076
WLLU                       18076
COD3_OSTXT_PA              18074
PRI_PAYMENT_CTRY_TCR_PA    18074
                           ...  
ORGAN                          0
CMV_IGG                        0
CMV_IGM                        0
EBV_SEROSTATUS                 0
index                          0
Length: 431, dtype: int64

In [9]:
df_10.shape

(18077, 431)


#### Run a dynamic sql queery giving me the percentage of nulls per each column in a table.

In [10]:
# Kidpan Data
columns = list(df_10)
kidpan_nulls = pd.DataFrame()

for col in columns:
    
    query = f"""SELECT \'{col}\' as column, 1-CAST(COUNT("{col}") as float)/CAST(COUNT(*) as float) as percent_null FROM kidpan_data;"""
    tmp_table = pd.read_sql_query(query, conn)
    kidpan_nulls = pd.concat([kidpan_nulls, tmp_table])                                  

In [11]:
kidpan_nulls.to_csv('../data/kidpan_nulls.csv', index = False)

In [13]:
kidpan_nulls = pd.read_csv('../data/kidpan_nulls.csv')
kidpan_nulls.head()

,column,percent_null
0,index,0.000000
1,WL_ORG,0.045441
2,NUM_PREV_TX,0.045441
3,CREAT_CLEAR,0.954463
4,GFR,0.733060


In [14]:
kidpan_nulls.set_index('column', inplace = True)
kidpan_nulls.head()

,percent_null
column,
index,0.000000
WL_ORG,0.045441
NUM_PREV_TX,0.045441
CREAT_CLEAR,0.954463
GFR,0.733060


In [16]:
pd.set_option('display.max_rows', 450)
kidpan_nulls['percent_null'].sort_values(ascending = False)

column
PRI_PAYMENT_CTRY_TRR_PA            0.999947
PRI_PAYMENT_CTRY_TCR_PA            0.999945
TXLNG                              0.999907
COD3_OSTXT_PA                      0.999899
WLLU                               0.999824
COD3_PA                            0.999779
BLOOD_SUGAR_MED_RESUMED_DATE_PA    0.999758
BLOOD_SUGAR_DIET_PA                0.999666
COD2_OSTXT_PA                      0.999642
BLOOD_SUGAR_MEDICATION_PA          0.999586
PRI_PAYMENT_CTRY_TCR_KI            0.999537
PRI_PAYMENT_CTRY_TRR_KI            0.999494
WLPI                               0.999457
COD2_PA                            0.999264
MALIG_OSTXT_TRR                    0.999157
OTH_GRF_FAIL_CAUSE_OSTXT_PA        0.999012
DGN2_OSTXT_TCR                     0.998799
COD3_OSTXT_KI                      0.998596
DUCT_MGMT_OSTXT                    0.998592
TXINT                              0.998587
OTH_COMPL_OSTXT                    0.998460
DIAG_OSTXT_PA                      0.998447
ENTERIC_DRAIN_DT         

In [ ]:
# Kidney Followup Data
kid_followup_nulls = pd.DataFrame()

for col in columns:
    
    query = f"""SELECT \'{col}\' as column, 1-CAST(COUNT("{col}") as float)/CAST(COUNT(*) as float) as percent_null FROM kidney_followup_data;"""
    tmp_table = pd.read_sql_query(query, conn)
    print(f"{col}")
    kid_followup_nulls = pd.concat([kid_followup_nulls, tmp_table]) 

In [ ]:
# Deceased Donor Data
dec_donor_nulls = pd.DataFrame()

for col in columns:
    
    query = f"""SELECT \'{col}\' as column, 1-CAST(COUNT("{col}") as float)/CAST(COUNT(*) as float) as percent_null FROM deceased_donor_data;"""
    tmp_table = pd.read_sql_query(query, conn)
    print(f"{col}")
    dec_donor_nulls = pd.concat([dec_donor_nulls, tmp_table]) 